# Troppo

In [1]:
import pandas as pd
import numpy as np
import cobra
import re
import pickle
import dill
import seaborn as sns

from troppo.omics.readers.generic import TabularReader
from troppo.methods_wrappers import ModelBasedWrapper, ReconstructionWrapper
from troppo.omics.integration import ContinuousScoreIntegrationStrategy
from troppo.methods.reconstruction.gimme import GIMME, GIMMEProperties

/project/Fish_heart_regeneration/jying/miniconda3/envs/troppo/lib/python3.10/site-packages/cobamp/wrappers/external_wrappers.py:9: UserWarning: 
The wrappers.external_wrappers module will be deprecated in a future release in favour of the wrappers module. 
    Available ModelObjectReader classes can still be loaded using cobamp.wrappers.<class>. An appropriate model 
    reader can also be created using the get_model_reader function on cobamp.wrappers
  warnings.warn(


In [2]:
# define the parsing rules for the GPRs that will be used later on
patt = re.compile('__COBAMPGPRDOT__[0-9]{1}')
replace_alt_transcripts = lambda x: patt.sub('', x)

In [3]:
# load model
import lxml
model = cobra.io.read_sbml_model('model/Zebrafish-GEM.xml')

https://identifiers.org/taxonomy/ does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id


In [4]:
# Save the object to a file
# with open('ZebrafishGEM.pkl', 'wb') as f:
#     pickle.dump(model, f)

In [5]:
# with open('ZebrafishGEM.pkl', 'rb') as f:
#     model = pickle.load(f)

In [4]:
model

Name,ZebrafishGEM
Memory address,7fd991857820
Number of metabolites,8439
Number of reactions,12809
Number of genes,2705
Number of groups,152
Objective expression,1.0*MAR00021 - 1.0*MAR00021_reverse_97974
Compartments,"Cytosol, Extracellular, Lysosome, Endoplasmic reticulum, Mitochondria, Peroxisome, Golgi apparatus, Nucleus, Inner mitochondria"


In [5]:
model.genes[0:5] # model genes in symbol

[<Gene LOC101886429 at 0x7fd9909d2c80>,
 <Gene aaas at 0x7fd9909d2b30>,
 <Gene aacs at 0x7fd9909d2ad0>,
 <Gene aadac at 0x7fd9909d2a10>,
 <Gene aadat at 0x7fd9909d2b60>]

## Use Raw Counts

In [9]:
# load data
expression_data = pd.read_table('fish_strains_project2_raw_counts.txt', index_col=1)
expression_data = expression_data.drop(columns=['id','description','Unnamed: 64'])

In [10]:
expression_data.head()

,AB_1Dpci_21,AB_1Dpci_22,AB_1Dpci_23,AB_7Dpci_24,AB_7Dpci_27,AB_7Dpci_5,AB_unop_4,AB_unop_6,AB_unop_7,KCL_1Dpci_62,...,TU_unop_3,WIK_1Dpci_55,WIK_1Dpci_56,WIK_1Dpci_57,WIK_7Dpci_58,WIK_7Dpci_60,WIK_7Dpci_61,WIK_unop_1,WIK_unop_2,WIK_unop_3
symbol,,,,,,,,,,,,,,,,,,,,,
CR383668.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
fgfr1op2,39,49,22,21,57,45,32,30,23,28,...,32,53,25,55,16,24,56,36,28,32
AL845295.2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
si:dkey-21h14.12,1,1,0,4,0,3,0,2,0,0,...,0,0,0,0,0,0,0,1,0,0
si:dkey-285e18.2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Create a model wrapper

In [ ]:
model_wrapper = ReconstructionWrapper(model=model, ttg_ratio=9999,
                                      gpr_gene_parse_function=replace_alt_transcripts,
                                     )
# ~1000 warnings (could not normalize 'rule')

## Map gene IDs in the data to model IDs

In [15]:
def score_apply(reaction_map_scores):
    return {k:0  if v is None else v for k, v in reaction_map_scores.items()}

In [16]:
omics_container = TabularReader(path_or_df=expression_data, nomenclature='external_gene_name',sample_in_rows=False,
                                omics_type='transcriptomics').to_containers()

In [36]:
omics_container[0].get_Condition()

'AB_1Dpci_21'

## Enrichment run

In [ ]:
modelname = 'gimme_1000X'
for x in range(len(omics_container)):

    single_sample = omics_container[x]
    
    data_map = single_sample.get_integrated_data_map(model_reader=model_wrapper.model_reader,
                                                     and_func=min, or_func=sum)
    continuous_integration = ContinuousScoreIntegrationStrategy(score_apply=score_apply)
    scores = continuous_integration.integrate(data_map=data_map)

    sampname = single_sample.get_Condition()
    scoredf = pd.DataFrame(scores.values(),index=scores.keys(),columns=[sampname])
    scoredf.to_csv(f'Files/scores_each_sample/scores_{modelname}_{sampname}.csv')

    if x==0:
        scoredf0 = scoredf
    else:
        scoredf0 = pd.concat([scoredf0,scoredf],axis=1)

In [37]:
scoredf0.head()

,AB_1Dpci_21,AB_1Dpci_22,AB_1Dpci_23,AB_7Dpci_24,AB_7Dpci_27,AB_7Dpci_5,AB_unop_4,AB_unop_6,AB_unop_7,KCL_1Dpci_62,...,TU_unop_3,WIK_1Dpci_55,WIK_1Dpci_56,WIK_1Dpci_57,WIK_7Dpci_58,WIK_7Dpci_60,WIK_7Dpci_61,WIK_unop_1,WIK_unop_2,WIK_unop_3
MAR03905,105,99,98,99,153,153,126,129,151,66,...,212,187,118,85,116,190,165,84,148,74
MAR03907,42,21,38,11,4,25,8,9,15,30,...,19,64,20,39,18,23,5,11,25,15
MAR04097,95,78,125,35,37,55,46,42,49,49,...,93,73,73,88,52,47,68,46,52,35
MAR04099,30,55,48,47,17,53,63,39,56,8,...,35,47,10,21,21,17,23,18,23,9
MAR04108,95,78,125,35,37,55,46,42,49,49,...,93,73,73,88,52,47,68,46,52,35


In [38]:
scoredf0.to_csv(f'Files/scores_{modelname}.csv')